# Categorical Variables
Категориальная переменная принимает только ограниченное число значений

#### Подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("../datasets/melb_data.csv")

# Для примера удалим лишние столбцы
data = data.drop(['Suburb', 'Address', 'SellerG', 'Date', 'CouncilArea'], axis=1)

# Целевые значения
y = data.Price

# Отбросим колонку целевых значений из датасета
X = data.drop("Price", axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

In [3]:
# Получаем список всех категориальных переменных в обучающих данных
s = (X_train.dtypes == "object")
print(s, "\n")
object_cols = list(s[s].index)
object_cols

Rooms            False
Type              True
Method            True
Distance         False
Postcode         False
Bedroom2         False
Bathroom         False
Car              False
Landsize         False
BuildingArea     False
YearBuilt        False
Lattitude        False
Longtitude       False
Regionname        True
Propertycount    False
dtype: bool 



['Type', 'Method', 'Regionname']

## Подход 1
#### Устроить им экстерминатус (Удалить категориальные переменные)

In [4]:
drop_X_train = X_train.select_dtypes(exclude=["object"])
drop_X_test = X_test.select_dtypes(exclude=["object"])

## Подход 2
#### Порядковая кодировка
Порядковая кодировка присваивает каждому уникальному значению отдельное целое число.
Этот подход предполагает упорядочение категорий: "Никогда" (0) < "Редко" (1) < "Большинство дней" (2) < "Каждый день" (3).
##### Данный подход имеет смысл тоько при явном ражировании категорий как в примере выше.

In [5]:
from sklearn.preprocessing import OrdinalEncoder

In [6]:
#  Сделаем копию, чтобы избежать изменения исходных данных 
label_X_train = X_train.copy()
label_X_test = X_test.copy()

# Применим порядковый кодировщик к каждому столбцу с категориальными данными
ordinal_encoder = OrdinalEncoder()

label_X_train[object_cols] = ordinal_encoder.fit_transform(label_X_train[object_cols])
label_X_test[object_cols] = ordinal_encoder.fit_transform(label_X_test[object_cols])
label_X_train[["Type", "Method", "Regionname"]]

,Type,Method,Regionname
12796,0.0,1.0,0.0
9642,0.0,1.0,0.0
3207,2.0,1.0,5.0
1698,2.0,1.0,2.0
761,0.0,1.0,5.0
...,...,...,...
5191,0.0,1.0,2.0
13418,0.0,1.0,2.0
5390,1.0,0.0,2.0
860,0.0,1.0,5.0


## Подход 3
#### Однократное кодирование
Используем класс OneHotEncoder из sklearn для получения однократной кодировки. Существует ряд параметров, которые можно использовать для настройки его поведения

Устанавливаем handle_unknown='ignore', чтобы избежать ошибок, когда данные проверки содержат классы, которые не представлены в обучающих данных, и 
установка sparse=False гарантирует, что закодированные столбцы будут возвращены в виде массива numpy (вместо разреженной матрицы).
Чтобы использовать кодировщик, мы предоставляем только те категориальные столбцы, которые мы хотим иметь одноразовое кодирование. Например, для кодирования обучающих данных мы предоставляем X_train[object_cols] . (object_cols в ячейке кода ниже представляет собой список имен столбцов с категориальными данными, и поэтому X_train[object_cols] содержит все категориальные данные в обучающем наборе.)

In [7]:
from sklearn.preprocessing import OneHotEncoder

In [8]:
OH_encoder = OneHotEncoder(
    handle_unknown="ignore",
    sparse_output=False,
)

OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test[object_cols]))

# Возвращаем обратно индексы
OH_cols_train.index = X_train.index
OH_cols_test.index = X_test.index

# Удаляем категоральные столбцы из датасета
num_X_train = X_train.drop(object_cols, axis=1)
num_X_test = X_test.drop(object_cols, axis=1)

# Объединяем новые столбцы с датасетом
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)

# Убеждаемся, что все столбцы имеют строковый тип
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_test.columns = OH_X_test.columns.astype(str)
OH_X_train

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,...,6,7,8,9,10,11,12,13,14,15
12796,4,14.2,3149.0,4.0,2.0,2.0,695.0,160.0,1970.0,-37.86127,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9642,3,14.2,3149.0,3.0,1.0,2.0,810.0,NaN,NaN,-37.86838,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3207,2,4.6,3122.0,2.0,1.0,1.0,82.0,NaN,NaN,-37.81800,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1698,2,3.2,3054.0,2.0,1.0,1.0,0.0,76.0,1975.0,-37.79020,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
761,4,13.0,3204.0,4.0,2.0,1.0,292.0,NaN,NaN,-37.91480,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5191,3,11.2,3073.0,3.0,1.0,1.0,0.0,NaN,1930.0,-37.72170,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
13418,2,3.6,3068.0,2.0,1.0,1.0,164.0,93.0,1890.0,-37.77796,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5390,3,2.6,3121.0,3.0,1.0,1.0,383.0,NaN,1993.0,-37.82480,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
860,2,13.9,3165.0,2.0,2.0,1.0,201.0,79.0,2005.0,-37.92930,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
